## Testing Some BS

In [1]:
import sys, os

In [13]:
# this seems to work
sys.path.insert(0, '..')

In [3]:
import pandas as pd
import numpy as np
import datetime as dt

In [8]:
# import modules within the application
# fix issue with modules (unresolved)
from modules import findata as fd
import time_measurement as tm

In [9]:
class Error(Exception):
    pass

In [10]:
def getRiskFreeRateMonth():
    URL = "https://home.treasury.gov/resource-center/data-chart-center/interest-rates/TextView?type=daily_treasury_long_term_rate&field_tdr_date_value_month=202202"
    table = pd.read_html(URL)
    return table[0].iloc[:,[0,3]]

def getCurrentRiskFreeRate():
    data = getRiskFreeRateMonth()
    return np.double(data.iloc[-1,[1]])/100
    
getCurrentRiskFreeRate()

0.022099999999999998

In [12]:
# calculation with 64bits
def calculateWacc(industry='Total Market', country='United States', year=dt.date.today().year):
    # get risk free rate
    risk_free_rate = getCurrentRiskFreeRate()
    # get countries equity risk premium
    risk_premium = fd.getCountryERP(country)
    # todays unlevered beta 
    unlevered_beta = fd.getUnleveredBeta(industry,year)
    # cost of equity
    equity_cost = risk_premium * unlevered_beta + risk_free_rate
    post_tax_debt_cost = fd.getCostOfDebt()
    result = equity_cost + post_tax_debt_cost
    return result
calculateWacc()

RecursionError: maximum recursion depth exceeded

In [ ]:
type(calculateWacc())

In [ ]:
def calculateWacc32bit(industry='Total Market', country='United States', year=dt.date.today().year):
    # get risk free rate
    risk_free_rate = np.float32(getCurrentRiskFreeRate())
    # get countries equity risk premium
    risk_premium = np.float32(fd.getCountryERP(country))
    # todays unlevered beta 
    unlevered_beta = np.float32(fd.getUnleveredBeta(industry,year))
    # cost of equity
    equity_cost = risk_premium * unlevered_beta + risk_free_rate
    post_tax_debt_cost = np.float32(fd.getCostOfDebt())
    result = equity_cost + post_tax_debt_cost
    return result
calculateWacc32bit()

In [ ]:
type(calculateWacc32bit())

=> convert 64-bit into 32 bit, because 64 bit takes 2x more memory

How about computing time? Calcualtion with 32bit values should be faster than 64bit values? If, yes, how much faster?

In [75]:
# convering to 32-bit
# type casting the return value
# probably won't change alot
_32bit = np.float32(calculateWacc())

NameError: name 'getUinleveredBeta' is not defined

In [ ]:
type(_32bit)

### Measuring time with 64bit

In [76]:
time_calculateWacc = tm.calculate_time(calculateWacc)
time_calculateWacc()

NameError: name 'getUinleveredBeta' is not defined

### Measuring time with 32bit

In [ ]:
time_32bit = tm.calculate_time(calculateWacc32bit)
time_32bit()

## Solution for % in DValues

In [ ]:
message = " this    is   "
print(message.strip())

In [ ]:
message2 = "this     is"
" ".join(message2.split())

## Solution for Redundant Whitespaces in Datatables

In [ ]:
" ".join(message.split()) ## -> iterating over names and remove whitespace!! in data

In [ ]:
# testing whitspace removal

def removeWhitespace(dataframe, col_name):
    # iterate over dataframe
    # locate redundant whitespace in col "name"
    # remove whitespace with
    for i, row in dataframe.iterrows():
        " ".join(row[col_name].split())
    return dataframe

Whitespace removal complexity: O(n)

## Testing Whitespace removal

In [50]:
unlevered_data = fd.unleveredBeta()
unlevered_data

AttributeError: module 'findata' has no attribute 'unleveredBeta'

In [51]:
old_index = unlevered_data.index
print(old_index)

NameError: name 'unlevered_data' is not defined

## cleaning data

In [295]:
"Financial  Svcs. (Non-bank & Insuran" in old_index

True

In [332]:
new_index = []
for i in old_index:
    new_index.append(" ".join(i.split()))
new_index

['Advertising',
 'Aerospace/Defense',
 'Air Transport',
 'Apparel',
 'Auto & Truck',
 'Auto Parts',
 'Bank (Money Center)',
 'Banks (Regional)',
 'Beverage (Alcoholic)',
 'Beverage (Soft)',
 'Broadcasting',
 'Brokerage & Investment Banking',
 'Building Materials',
 'Business & Consumer Services',
 'Cable TV',
 'Chemical (Basic)',
 'Chemical (Diversified)',
 'Chemical (Specialty)',
 'Coal & Related Energy',
 'Computer Services',
 'Computers/Peripherals',
 'Construction Supplies',
 'Diversified',
 'Drugs (Biotechnology)',
 'Drugs (Pharmaceutical)',
 'Education',
 'Electrical Equipment',
 'Electronics (Consumer & Office)',
 'Electronics (General)',
 'Engineering/Construction',
 'Entertainment',
 'Environmental & Waste Services',
 'Farming/Agriculture',
 'Financial Svcs. (Non-bank & Insuran',
 'Food Processing',
 'Food Wholesalers',
 'Furn/Home Furnishings',
 'Green & Renewable Energy',
 'Healthcare Products',
 'Healthcare Support Services',
 'Heathcare Information and Technol',
 'Homebuil

In [333]:
"Financial Svcs. (Non-bank & Insuran" in new_index

True

In [334]:
# i need to write a dict
tuple_1 = [("Financial Svcs. (Non-bank & Insuran", "Financial Services (Non-bank  & Insurance)"),
          ("Healthcare Information and Technol","Healthcare Information and Technology"),
           ("Oil/Gas (Production and Exploratio", "Oil/Gas (Production and Exploration)"),
           ("Oilfield Svcs/Equip.", "Oilfield Services/Equipment"),
           ("R.E.I.T", "Real Estate Investment Trust"),
           ("Restaurant/Dining", "Restaurants"),
           ("Rubber& Tires", "Rubber & Tires"),
          ("Semiconductor Equip", "Semiconductor Equipment"),
          ("Software, System & Application", "Software System & Application"),
          ("Telecom (Wireless)", "Telecommunication (Wireless)"),
          ("Telecom. Equipment", "Telecommunication Equipment"),
          ("Telecom. Services", "Telecommunication Services"),
          ("Total Market (without financials)", "Total Market (excl. Financials)")]
tuple_1[0][0]
for i in range(len(tuple_1)):
    print("{} , {} ".format(tuple_1[i][0], tuple_1[i][1]))

Financial Svcs. (Non-bank & Insuran , Financial Services (Non-bank  & Insurance) 
Healthcare Information and Technol , Healthcare Information and Technology 
Oil/Gas (Production and Exploratio , Oil/Gas (Production and Exploration) 
Oilfield Svcs/Equip. , Oilfield Services/Equipment 
R.E.I.T , Real Estate Investment Trust 
Restaurant/Dining , Restaurants 
Rubber& Tires , Rubber & Tires 
Semiconductor Equip , Semiconductor Equipment 
Software, System & Application , Software System & Application 
Telecom (Wireless) , Telecommunication (Wireless) 
Telecom. Equipment , Telecommunication Equipment 
Telecom. Services , Telecommunication Services 
Total Market (without financials) , Total Market (excl. Financials) 


In [336]:
# new_index = [w.replace(tuple_1[i][0], tuple_1[i][1]) for w in new_index for i in range(len(tuple_1))]
for i in range(len(tuple_1)):
    new_index = [w.replace(tuple_1[i][0], tuple_1[i][1]) for w in new_index]
new_index

['Advertising',
 'Aerospace/Defense',
 'Air Transport',
 'Apparel',
 'Auto & Truck',
 'Auto Parts',
 'Bank (Money Center)',
 'Banks (Regional)',
 'Beverage (Alcoholic)',
 'Beverage (Soft)',
 'Broadcasting',
 'Brokerage & Investment Banking',
 'Building Materials',
 'Business & Consumer Services',
 'Cable TV',
 'Chemical (Basic)',
 'Chemical (Diversified)',
 'Chemical (Specialty)',
 'Coal & Related Energy',
 'Computer Services',
 'Computers/Peripherals',
 'Construction Supplies',
 'Diversified',
 'Drugs (Biotechnology)',
 'Drugs (Pharmaceutical)',
 'Education',
 'Electrical Equipment',
 'Electronics (Consumer & Office)',
 'Electronics (General)',
 'Engineering/Construction',
 'Entertainment',
 'Environmental & Waste Services',
 'Farming/Agriculture',
 'Financial Services (Non-bank  & Insurance)',
 'Food Processing',
 'Food Wholesalers',
 'Furn/Home Furnishings',
 'Green & Renewable Energy',
 'Healthcare Products',
 'Healthcare Support Services',
 'Heathcare Information and Technol',
 'H

In [325]:
new_index

['Advertising',
 'Aerospace/Defense',
 'Air Transport',
 'Apparel',
 'Auto & Truck',
 'Auto Parts',
 'Bank (Money Center)',
 'Banks (Regional)',
 'Beverage (Alcoholic)',
 'Beverage (Soft)',
 'Broadcasting',
 'Brokerage & Investment Banking',
 'Building Materials',
 'Business & Consumer Services',
 'Cable TV',
 'Chemical (Basic)',
 'Chemical (Diversified)',
 'Chemical (Specialty)',
 'Coal & Related Energy',
 'Computer Services',
 'Computers/Peripherals',
 'Construction Supplies',
 'Diversified',
 'Drugs (Biotechnology)',
 'Drugs (Pharmaceutical)',
 'Education',
 'Electrical Equipment',
 'Electronics (Consumer & Office)',
 'Electronics (General)',
 'Engineering/Construction',
 'Entertainment',
 'Environmental & Waste Services',
 'Farming/Agriculture',
 'Financial Services (Non-bank  & Insurance)',
 'Food Processing',
 'Food Wholesalers',
 'Furn/Home Furnishings',
 'Green & Renewable Energy',
 'Healthcare Products',
 'Healthcare Support Services',
 'Heathcare Information and Technol',
 'H

In [337]:
def cleaningBetaData(dataframe):
    beta_data = dataframe
    beta_data = beta_data.set_index('Industry Name')
    # remove whitespace in index
    old_index = beta_data.index
    new_index = []
    for i in old_index:
        new_index.append(" ".join(i.split()))
    # replace value in new_index
    tuple_1 = [("Financial Svcs. (Non-bank & Insuran", "Financial Services (Non-bank  & Insurance)"),
          ("Healthcare Information and Technol","Healthcare Information and Technology"),
           ("Oil/Gas (Production and Exploratio", "Oil/Gas (Production and Exploration)"),
           ("Oilfield Svcs/Equip.", "Oilfield Services/Equipment"),
           ("R.E.I.T", "Real Estate Investment Trust"),
           ("Restaurant/Dining", "Restaurants"),
           ("Rubber& Tires", "Rubber & Tires"),
          ("Semiconductor Equip", "Semiconductor Equipment"),
          ("Software, System & Application", "Software System & Application"),
          ("Telecom (Wireless)", "Telecommunication (Wireless)"),
          ("Telecom. Equipment", "Telecommunication Equipment"),
          ("Telecom. Services", "Telecommunication Services"),
          ("Total Market (without financials)", "Total Market (excl. Financials)")]
    for i in range(len(tuple_1)):
        new_index = [w.replace(tuple_1[i][0], tuple_1[i][1]) for w in new_index]
        
    # df.set_index([pd.Index([1, 2, 3, 4]), 'year'])
    beta_data = beta_data.set_index([pd.Index(new_index)])
    return beta_data

In [338]:
cleaningBetaData(fd.betaData())

,Number of firms,Beta,D/E Ratio,Effective Tax rate,Unlevered beta,Cash/Firm value,Unlevered beta corrected for cash,HiLo Risk,Standard deviation of equity,Standard deviation in operating income (last 10 years),2018.0,2019.0,2020.0,2021.0,Average (2017-22)
Advertising,49,1.34,51.47%,5.76%,0.97,11.57%,1.10,0.6674,56.70%,13.52%,0.78,0.87,0.93,0.77,0.89
Aerospace/Defense,73,1.28,29.45%,6.83%,1.05,5.05%,1.11,0.4765,38.23%,19.61%,0.99,1.09,1.08,0.91,1.04
Air Transport,21,1.58,153.33%,5.32%,0.75,18.30%,0.91,0.4020,40.19%,162.40%,0.67,0.63,0.84,0.91,0.80
Apparel,39,1.23,31.60%,12.06%,1.00,9.26%,1.10,0.4896,43.49%,21.42%,0.85,0.76,0.83,0.94,0.90
Auto & Truck,26,1.13,19.86%,3.88%,0.99,3.38%,1.02,0.5955,54.78%,35.30%,0.59,0.34,0.53,1.05,0.71
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Trucking,34,1.44,26.26%,16.04%,1.21,5.57%,1.28,0.4574,32.99%,148.80%,0.81,0.71,1.04,0.94,0.96
Utility (General),16,0.89,69.22%,9.75%,0.59,0.63%,0.60,0.1395,18.83%,8.86%,0.20,0.17,0.19,0.48,0.33
Utility (Water),14,0.77,34.34%,10.01%,0.61,0.37%,0.61,0.3152,41.01%,13.97%,0.27,0.32,0.57,0.57,0.47
Total Market,7229,1.09,40.09%,7.05%,0.85,6.87%,0.91,0.4659,41.01%,10.28%,0.72,0.80,0.83,0.75,0.80


In [141]:
beta_data = fd.betaData()
beta_data.set_index("Industry Name")

,Number of firms,Beta,D/E Ratio,Effective Tax rate,Unlevered beta,Cash/Firm value,Unlevered beta corrected for cash,HiLo Risk,Standard deviation of equity,Standard deviation in operating income (last 10 years),2018.0,2019.0,2020.0,2021.0,Average (2017-22)
Industry Name,,,,,,,,,,,,,,,
Advertising,49,1.34,51.47%,5.76%,0.97,11.57%,1.10,0.6674,56.70%,13.52%,0.78,0.87,0.93,0.77,0.89
Aerospace/Defense,73,1.28,29.45%,6.83%,1.05,5.05%,1.11,0.4765,38.23%,19.61%,0.99,1.09,1.08,0.91,1.04
Air Transport,21,1.58,153.33%,5.32%,0.75,18.30%,0.91,0.4020,40.19%,162.40%,0.67,0.63,0.84,0.91,0.80
Apparel,39,1.23,31.60%,12.06%,1.00,9.26%,1.10,0.4896,43.49%,21.42%,0.85,0.76,0.83,0.94,0.90
Auto & Truck,26,1.13,19.86%,3.88%,0.99,3.38%,1.02,0.5955,54.78%,35.30%,0.59,0.34,0.53,1.05,0.71
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Trucking,34,1.44,26.26%,16.04%,1.21,5.57%,1.28,0.4574,32.99%,148.80%,0.81,0.71,1.04,0.94,0.96
Utility (General),16,0.89,69.22%,9.75%,0.59,0.63%,0.60,0.1395,18.83%,8.86%,0.20,0.17,0.19,0.48,0.33
Utility (Water),14,0.77,34.34%,10.01%,0.61,0.37%,0.61,0.3152,41.01%,13.97%,0.27,0.32,0.57,0.57,0.47


## more sophisticated whitespace removal

In [103]:
new_index = []
for i in unlevered_data.index:
    new_index.append(" ".join(i.split()))
print(new_index)

['Advertising', 'Aerospace/Defense', 'Air Transport', 'Apparel', 'Auto & Truck', 'Auto Parts', 'Bank (Money Center)', 'Banks (Regional)', 'Beverage (Alcoholic)', 'Beverage (Soft)', 'Broadcasting', 'Brokerage & Investment Banking', 'Building Materials', 'Business & Consumer Services', 'Cable TV', 'Chemical (Basic)', 'Chemical (Diversified)', 'Chemical (Specialty)', 'Coal & Related Energy', 'Computer Services', 'Computers/Peripherals', 'Construction Supplies', 'Diversified', 'Drugs (Biotechnology)', 'Drugs (Pharmaceutical)', 'Education', 'Electrical Equipment', 'Electronics (Consumer & Office)', 'Electronics (General)', 'Engineering/Construction', 'Entertainment', 'Environmental & Waste Services', 'Farming/Agriculture', 'Financial Svcs. (Non-bank & Insuran', 'Food Processing', 'Food Wholesalers', 'Furn/Home Furnishings', 'Green & Renewable Energy', 'Healthcare Products', 'Healthcare Support Services', 'Heathcare Information and Technol', 'Homebuilding', 'Hospitals/Healthcare Facilities',

In [108]:

beta_data.index.is_integer()

True

In [122]:
beta_data

,Industry Name,Number of firms,Beta,D/E Ratio,Effective Tax rate,Unlevered beta,Cash/Firm value,Unlevered beta corrected for cash,HiLo Risk,Standard deviation of equity,Standard deviation in operating income (last 10 years),2018.0,2019.0,2020.0,2021.0,Average (2017-22)
1,Advertising,49,1.34,51.47%,5.76%,0.97,11.57%,1.10,0.6674,56.70%,13.52%,0.78,0.87,0.93,0.77,0.89
2,Aerospace/Defense,73,1.28,29.45%,6.83%,1.05,5.05%,1.11,0.4765,38.23%,19.61%,0.99,1.09,1.08,0.91,1.04
3,Air Transport,21,1.58,153.33%,5.32%,0.75,18.30%,0.91,0.4020,40.19%,162.40%,0.67,0.63,0.84,0.91,0.80
4,Apparel,39,1.23,31.60%,12.06%,1.00,9.26%,1.10,0.4896,43.49%,21.42%,0.85,0.76,0.83,0.94,0.90
5,Auto & Truck,26,1.13,19.86%,3.88%,0.99,3.38%,1.02,0.5955,54.78%,35.30%,0.59,0.34,0.53,1.05,0.71
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92,Trucking,34,1.44,26.26%,16.04%,1.21,5.57%,1.28,0.4574,32.99%,148.80%,0.81,0.71,1.04,0.94,0.96
93,Utility (General),16,0.89,69.22%,9.75%,0.59,0.63%,0.60,0.1395,18.83%,8.86%,0.20,0.17,0.19,0.48,0.33
94,Utility (Water),14,0.77,34.34%,10.01%,0.61,0.37%,0.61,0.3152,41.01%,13.97%,0.27,0.32,0.57,0.57,0.47
95,Total Market,7229,1.09,40.09%,7.05%,0.85,6.87%,0.91,0.4659,41.01%,10.28%,0.72,0.80,0.83,0.75,0.80


In [339]:
unlevered_data.index.is_integer()

False

In [340]:
def f(*args):
    print(list(args))

f(1,2,3)

[1, 2, 3]


In [341]:
beta_data[("Number of firms", "Beta")]

KeyError: ('Number of firms', 'Beta')

In [342]:
def printCol(dataframe, *col_name):
    for i, row in dataframe.iterrows():
        print(row[col_name])
printCol(beta_data, ("Number of firms", "Beta") )

KeyError: 'key of type tuple not found and not a MultiIndex'

In [365]:
def removeWhitespaceInIndex(dataframe):
    # first we check if the index is a string or an integer or float
    if ( (dataframe.index.is_integer or dataframe.index.is_float) == True):
        raise Error("Index should contain string values")
    else:
        # iterate over dataframe
        # locate redundant whitespace in col "name"
        # remove whitespace with
        for i, row in dataframe.iterrows():
            " ".join(i.split())
    return dataframe

In [366]:
removeWhitespaceInIndex(fd.unleveredBeta())

,2022,2021,2020,2019,2018,2017
Industry Name,,,,,,
Advertising,0.97,0.89,0.77,0.93,0.87,0.78
Aerospace/Defense,1.05,1.04,0.91,1.08,1.09,0.99
Air Transport,0.75,0.80,0.91,0.84,0.63,0.67
Apparel,1.00,0.90,0.94,0.83,0.76,0.85
Auto & Truck,0.99,0.71,1.05,0.53,0.34,0.59
...,...,...,...,...,...,...
Trucking,1.21,0.96,0.94,1.04,0.71,0.81
Utility (General),0.59,0.33,0.48,0.19,0.17,0.2
Utility (Water),0.61,0.47,0.57,0.57,0.32,0.27


In [359]:
fd.unleveredBeta().index.

,2022,2021,2020,2019,2018,2017
Industry Name,,,,,,
Advertising,0.97,0.89,0.77,0.93,0.87,0.78
Aerospace/Defense,1.05,1.04,0.91,1.08,1.09,0.99
Air Transport,0.75,0.80,0.91,0.84,0.63,0.67
Apparel,1.00,0.90,0.94,0.83,0.76,0.85
Auto & Truck,0.99,0.71,1.05,0.53,0.34,0.59
...,...,...,...,...,...,...
Trucking,1.21,0.96,0.94,1.04,0.71,0.81
Utility (General),0.59,0.33,0.48,0.19,0.17,0.2
Utility (Water),0.61,0.47,0.57,0.57,0.32,0.27
